# 3-5 Pyramid Pooling モジュールの解説と実装

## Pyramid Pooling モジュールのサブネットワーク構造
Pyramid Pooling モジュールの構造を次に示す．

<img src="../image/p158.png">

モジュールへの入力は Feature モジュールから出力された 2048 x 60 x 60 のテンソルである．
この入力が5つに分岐する．
一番上の分岐は Adaptive Average Pooling 層（出力=6）に送られる．
この層では画像の（嵩さ×幅）に対して出力で指定したサイズの大きさに変換されるように Average Pooling を行う．
つまり，60 x 60 の入力が 6 x 6 に変換される．
同様にして，5つの分岐のうち4つはそれぞれ出力が6，3，2，1となるような Adaptive Average Pooling 層に入力される．
このように出力サイズの異なる Average Pooling 層を用いることで，入力画像に対して様々なサイズの特徴量の処理（マルチスケール処理）を実現している．  
Average Pooling 層を通過したテンソルは，conv2DBatchNormRelu クラスを経て最後に UpSample 層に到達する．
UpSample 層では bilinear 処理によって 小さくなった画像をもとの 60 x 60 まで引き伸ばす．  
残り1つの分岐は入力をそのまま出力に渡し他の4つの分岐と結合させる．
4つの分岐の出力はそれぞれ 512 チャネルで，全ての出力を結合することで最終的な出力は 512 × 4 ＋ 2048 = 4096 チャネルとなる．
このような処理を施すことで，マルチスケールな情報を得ることができるため，高い精度でセマンティックセグメンテーションを実現できる．

## クラス PyramidPooling の実装
PyramidPooling クラスでは入力を5つに分岐させ、Adaptive Average Pooling 層、conv2DBatchNormRelu、UpSample 層を通過させて最後に 1 つのテンソルに再結合させる．

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class PyramidPooling(nn.Module):
    def __init__(self, in_channels, pool_sizes, height, width):
        super(PyramidPooling, self).__init__()
        
        # forward で使用する画像サイズ
        self.height = height
        self.width = width
        
        # 各畳み込み層の出力チャンネル数
        out_channels = int(in_channels / len(pool_sizes))
        
        # 各畳み込み層を作成
        self.avpool_1 = nn.AdaptiveAvgPool2d(output_size=pool_sizes[0])
        self.cbr_1 = conv2DBatchNormRelu(in_channels, out_channels, kernel_size=1, stride=1, padding=0, dilation=1, bias=False)
        
        self.avpool_2 = nn.AdaptiveAvgPool2d(output_size=pool_sizes[1])
        self.cbr_2 = conv2DBatchNormRelu(in_channels, out_channels, kernel_size=1, stride=1, padding=0, dilation=1, bias=False)
        
        self.avpool_3 = nn.AdaptiveAvgPool2d(output_size=pool_sizes[2])
        self.cbr_3 = conv2DBatchNormRelu(in_channels, out_channels, kernel_size=1, stride=1, padding=0, dilation=1, bias=False)
        
        self.avpool_4 = nn.AdaptiveAvgPool2d(output_size=pool_sizes[3])
        self.cbr_4 = conv2DBatchNormRelu(in_channels, out_channels, kernel_size=1, stride=1, padding=0, dilation=1, bias=False)
        
        
    def forward(self, x):
        out1 = self.cbr_1(self.avpool_1(x))
        out1 = F.interpolate(out1, size=(self.height, self.width), mode="bilinear", align_corners=True)
        
        out2 = self.cbr_2(self.avpool_2(x))
        out2 = F.interpolate(out2, size=(self.height, self.width), mode="bilinear", align_corners=True)
        
        out3 = self.cbr_3(self.avpool_3(x))
        out3 = F.interpolate(out3, size=(self.height, self.width), mode="bilinear", align_corners=True)
        
        out4 = self.cbr_4(self.avpool_4(x))
        out4 = F.interpolate(out4, size=(self.height, self.width), mode="bilinear", align_corners=True)
        
        # 全ての出力を結合する
        output = torch.cat([x, out1, out2, out3, out4], dim=1)
        
        return output